# CASE 1: Static HWP Cycles - Intensities Matrix Inversion (No Noise)

In [1]:
import numpy as np
import sys
sys.path.insert(0, '../python/')
import angles
import instrument_mm
import stokes

# Define observer parameters
observer_latitude = 20.0  # Latitude of Mauna Kea in degrees
observer_longitude = -155.5  # Longitude of Mauna Kea in degrees

# Define target parameters
targets = [
    {"ra": 45.0, "dec": -50.0},  # RA 3h (45°), Dec -50°
    {"ra": 45.0, "dec": -20.0},  # RA 3h (45°), Dec -20°
    {"ra": 45.0, "dec": 15.0},   # RA 3h (45°), Dec 15°
]

# Polarization levels to inject (0.01%, 0.1%, and 1%)
polarization_levels = [0.0001, 0.001, 0.01]  # Convert percentages to decimal form

# Observation date and time
observation_date = "2460601.5"  # JD for 10/31/2024
observation_time = "14:00:00"   # UT time to get peak at midnight HST

# Run the simulation
for target in targets:
    print("*************************")
    print(f"Target RA: {target['ra']}°, Dec: {target['dec']}°")
    
    for pol in polarization_levels:
        # Inject Q and U polarization levels
        Q_injected = pol
        U_injected = 0
        injected_stokes = np.array([1, Q_injected, U_injected, 0])
        
        # Calculate the parallactic angle
        pa = angles.calculate_parallactic_angle(
            ra=target["ra"], 
            dec=target["dec"], 
            ut=observation_time, 
            jd_str=observation_date, 
            observer_latitude=observer_latitude, 
            observer_longitude=observer_longitude
        )
        
        # Calculate the altitude for each target at the given date and time
        H = angles.calculate_hour_angle(
            ra=target["ra"], 
            observer_longitude=observer_longitude, 
            ut=observation_time, 
            jd_str=observation_date
        )
        altitude = angles.calculate_altitude(
            phi=observer_latitude, 
            delta=target["dec"], 
            H=H
        )
        
        # Calculate the inverted input Stokes vector S_in using the new function
        S_in_retrieved = instrument_mm.calculate_input_Q_U_intensities_matrix_inversion(
            Q=Q_injected, 
            U=U_injected, 
            pa=pa, 
            altitude=altitude
        )
        
        # Calculate the absolute difference between injected and retrieved Stokes vectors
        absolute_difference = np.abs(injected_stokes - S_in_retrieved)
        
        # Output results
        print(f"Injected Stokes Vector: {injected_stokes}")
        print(f"Retrieved S_in: {S_in_retrieved}")
        print(f"Absolute Difference: {absolute_difference}\n")


*************************
Target RA: 45.0°, Dec: -50.0°
Injected Stokes Vector: [1.e+00 1.e-04 0.e+00 0.e+00]
Retrieved S_in: [1.00000000e+00 1.00000000e-04 1.77635684e-15 3.55271368e-15]
Absolute Difference: [4.44089210e-16 2.11026400e-16 1.77635684e-15 3.55271368e-15]

Injected Stokes Vector: [1.    0.001 0.    0.   ]
Retrieved S_in: [1.00000000e+00 1.00000000e-03 1.77635684e-15 7.10542736e-15]
Absolute Difference: [3.33066907e-16 3.33934269e-16 1.77635684e-15 7.10542736e-15]

Injected Stokes Vector: [1.   0.01 0.   0.  ]
Retrieved S_in: [1.00000000e+00 1.00000000e-02 1.77635684e-15 3.55271368e-15]
Absolute Difference: [2.22044605e-16 4.52762827e-16 1.77635684e-15 3.55271368e-15]

*************************
Target RA: 45.0°, Dec: -20.0°
Injected Stokes Vector: [1.e+00 1.e-04 0.e+00 0.e+00]
Retrieved S_in: [1.00000000e+00 1.00000000e-04 8.88178420e-16 1.77635684e-15]
Absolute Difference: [2.22044605e-16 5.26515680e-17 8.88178420e-16 1.77635684e-15]

Injected Stokes Vector: [1.    0.001

# CASE 2: Q and U Observing Sequence - Intensities Matrix Inversion

In [2]:
import numpy as np
import sys
sys.path.insert(0, '../python/')
import angles
import instrument_mm
import stokes

# Define observer parameters
observer_latitude = 20.0  # Latitude of Mauna Kea in degrees
observer_longitude = -155.5  # Longitude of Mauna Kea in degrees

# Define target parameters
targets = [
    {"ra": 45.0, "dec": -50.0},  # RA 3h (45°), Dec -50°
    {"ra": 45.0, "dec": -20.0},  # RA 3h (45°), Dec -20°
    {"ra": 45.0, "dec": 15.0},   # RA 3h (45°), Dec 15°
]

# Polarization levels to inject (0.01%, 0.1%, and 1%)
polarization_levels = [0.0001, 0.001, 0.01]  # Convert percentages to decimal form

# Observation date and time
observation_date = "2460601.5"  # JD for 10/31/2024
observation_time = "14:00:00"   # UT time to get peak at midnight HST
t_int = 60  # Integration time in seconds (modifiable variable)

# Run the simulation
for target in targets:
    print("*************************")
    print(f"Target RA: {target['ra']}°, Dec: {target['dec']}°")
    
    for pol in polarization_levels:
        # Inject Q and U polarization levels
        Q_injected = pol
        U_injected = 0
        injected_stokes = np.array([1, Q_injected, U_injected, 0])
        
        # Use the modified function with dynamic PA and altitude
        S_in_retrieved = instrument_mm.calculate_input_Q_U_observing_sequence_matrix_inversion(
            ra=target["ra"], 
            dec=target["dec"], 
            observer_latitude=observer_latitude, 
            observer_longitude=observer_longitude, 
            jd_str=observation_date, 
            ut_start=observation_time, 
            t_int=t_int, 
            Q=Q_injected, 
            U=U_injected, 
            delta_HWP=0.5
        )
        
        # Calculate the absolute difference between injected and retrieved Stokes vectors
        absolute_difference = np.abs(injected_stokes - S_in_retrieved)
        
        # Output results
        print(f"Injected Stokes Vector: {injected_stokes}")
        print(f"Retrieved S_in: {S_in_retrieved}")
        print(f"Absolute Difference: {absolute_difference}\n")


*************************
Target RA: 45.0°, Dec: -50.0°
Injected Stokes Vector: [1.e+00 1.e-04 0.e+00 0.e+00]
Retrieved S_in: [ 1.0000000e+00  1.0000000e-04 -8.8817842e-16  0.0000000e+00]
Absolute Difference: [0.00000000e+00 4.55107414e-16 8.88178420e-16 0.00000000e+00]

Injected Stokes Vector: [1.    0.001 0.    0.   ]
Retrieved S_in: [ 1.0000000e+00  1.0000000e-03 -8.8817842e-16  0.0000000e+00]
Absolute Difference: [0.00000000e+00 3.32199546e-16 8.88178420e-16 0.00000000e+00]

Injected Stokes Vector: [1.   0.01 0.   0.  ]
Retrieved S_in: [ 1.00000000e+00  1.00000000e-02 -1.77635684e-15  0.00000000e+00]
Absolute Difference: [0.00000000e+00 2.13370988e-16 1.77635684e-15 0.00000000e+00]

*************************
Target RA: 45.0°, Dec: -20.0°
Injected Stokes Vector: [1.e+00 1.e-04 0.e+00 0.e+00]
Retrieved S_in: [1.00000000e+00 1.00000000e-04 8.88178420e-16 3.55271368e-15]
Absolute Difference: [4.44089210e-16 8.04071436e-17 8.88178420e-16 3.55271368e-15]

Injected Stokes Vector: [1.    0